In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install openai
!pip install opencc-python-reimplemented
!pip install tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 31.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 20.7 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 481.8/481.8 kB 9.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
path = r"/content/drive/MyDrive/CSD_dataset.xlsx"
df = pd.read_excel(path)
df.info()

In [ ]:
import os
import openai
openai.api_key = "your-api-key"
openai.Completion.create(
  model="text-davinci-003",
  prompt="Say this is a test",
  max_tokens=5,
  temperature=0
)

In [ ]:
from opencc import OpenCC
cc = OpenCC('s2hk')
text = 'PLACEHOLDER'
prompt = "判断这条文本对疫苗的立场是支持，反对，或者其他。\n文本：" + text + "\n立场："
prompt = cc.convert(prompt)
print(prompt)

In [ ]:
from opencc import OpenCC
cc = OpenCC('s2hk')
idx =100
text = df['text'].iloc[idx]
prompt = "判断这条文本对疫苗的立场是支持，反对，或者其他。\n文本：" + text + "\n立场："
prompt = cc.convert(prompt)
print(prompt)

In [ ]:
import os
import openai
import time
from tqdm.notebook import tqdm
df = df2

df['chatgpt_label'] = ''
for i in tqdm(range(len(df))):
  text = df['text'].iloc[i]
  prompt = "判断这条文本对疫苗的立场是支持，反对，或者其他。\n文本：" + text + "\n立场："
  prompt = cc.convert(prompt)
  try:
      result = openai.Completion.create(
              model="text-davinci-003",
              prompt = prompt,
              max_tokens = 5,
              temperature=0)
  except:
      time.sleep(1)
      result = openai.Completion.create(
      model="text-davinci-003",
      prompt = prompt,
      max_tokens = 5,
      temperature=0)
  result = result["choices"][0]["text"].strip()
  df['chatgpt_label'].iloc[i] = result

In [ ]:
df.to_excel("resultgbt.xlsx", index = None)

In [ ]:
import pandas as pd
df =pd.read_excel("resultgbt.xlsx")

In [ ]:
df['chatgpt_label'].value_counts()

In [ ]:
df['chatgpt_label'] = df['chatgpt_label'].map({'其他':2, '支': 0, '反对':1, '其他。':2})
df['chatgpt_label'].value_counts()

In [ ]:
f1s = []
for i in range(100):
  test = df.sample(frac = 0.2, random_state = i)
  labels = test.label.to_list()
  preds = test.chatgpt_label.to_list()
  y_true = labels
  y_pred = preds
  f1 = f1_score(y_true, y_pred, average='macro')
  f1s.append(f1)

a_mean = np.mean(f1s)
a_std = np.std(f1s,ddof=1)

print("MEAN:", a_mean)
print("STD:", a_std)
print()